# 00 - Preparación y Agrupación de Datos (Pandas)

**Objetivo del notebook**: construir un flujo continuo que conecte (1) la exploración y preparación del dataset **Titanic** con (2) técnicas de **agrupamiento y pivoteo** en Pandas.

A lo largo del notebook trabajaremos **siempre con** `titanic.csv`.

---


## 1. Carga del dataset y primeras validaciones

Antes de transformar o agregar variables, validamos que el archivo esté accesible y que los tipos de datos tengan sentido.


---
# Series y DataFrames

- Lectura de datos
- Métodos básicos de exploración
- La estructura Serie
- La estructura DataFrame
- Selección de subset de datos
- Operacione estadísticas
- Filtrado de datos
- Creación de columnas en un dataframe


In [ ]:
import pandas as pd

#### Leemos los datos desde un archivo

In [ ]:
df = pd.read_csv('titanic.csv')

In [ ]:
# Ajustes de visualización (opcional)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)


In [ ]:
df

| Variable    | Traducción al Español     | Descripción                                                           |
| ----------- | ------------------------- | --------------------------------------------------------------------- |
| PassengerId | ID Pasajero               | Identificador único del pasajero                                      |
| Survived    | Sobrevivió                | Indica si el pasajero sobrevivió (0 = No, 1 = Sí)                     |
| Pclass      | Clase del Pasaje          | Clase del ticket (1 = Primera, 2 = Segunda, 3 = Tercera)              |
| Name        | Nombre                    | Nombre completo del pasajero                                          |
| Sex         | Sexo                      | Sexo del pasajero                                                     |
| Age         | Edad                      | Edad del pasajero en años                                             |
| SibSp       | Hermanos/Cónyuges a Bordo | Número de hermanos y/o cónyuges a bordo                               |
| Parch       | Padres/Hijos a Bordo      | Número de padres y/o hijos a bordo                                    |
| Ticket      | Número de Ticket          | Código del ticket                                                     |
| Fare        | Tarifa                    | Precio pagado por el pasaje                                           |
| Cabin       | Cabina                    | Identificador de la cabina                                            |
| Embarked    | Puerto de Embarque        | Puerto donde embarcó (C = Cherbourg, Q = Queenstown, S = Southampton) |


## 2. Exploración rápida para entender el dato

En esta sección usamos métodos básicos de Pandas para responder preguntas iniciales:
- ¿Cuántas filas/columnas tenemos?
- ¿Qué tipos de variables existen?
- ¿Hay valores faltantes?

Estas validaciones son clave porque condicionan **cómo** vamos a agrupar y resumir más adelante.


#### Métodos básicos de exploración de un DataFrame

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
df.describe(include='all')

In [ ]:
df.info()

#### La Serie

Seleccionando una columna del DataFrame

In [ ]:
df['Name']

Seleccionando una Fila del DataFrame

In [ ]:
df.iloc[0]

#### Seleccionando columnas de un DataFrame

In [ ]:
df[ ['Name','Age'] ]

#### Seleccionando filas de un DataFrame

In [ ]:
df.iloc[5:8]

In [ ]:
df.iloc[ [5,7,17] ]

#### Seleccionando celdas

In [ ]:
df.loc[2:5, 'Name']

In [ ]:
df.loc[2:5, ['Name','Age']]

#### Realizando Cálculos Estadísticos en una Serie o DataFrame

In [ ]:
df.min()

In [ ]:
df.select_dtypes(include="number").min()

In [ ]:
df['Fare'].min()

In [ ]:
df.select_dtypes(include="number").max()

In [ ]:
df['Fare'].max()

In [ ]:
df.count()

In [ ]:
df['Fare'].count()

In [ ]:
df.select_dtypes(include="number").median()

In [ ]:
df['Fare'].median()

In [ ]:
df.select_dtypes(include="number").mean()

In [ ]:
df['Fare'].mean()

In [ ]:
df.select_dtypes(include="number").quantile(q=0.1)

In [ ]:
df['Fare'].quantile(q=0.1)

In [ ]:
df.select_dtypes(include="number").quantile(q=0.5)

In [ ]:
df['Fare'].quantile(q=0.5)

In [ ]:
df.select_dtypes(include="number").quantile(q=0.9)

In [ ]:
df['Fare'].quantile(q=0.9)

#### Filtrando Filas de un DataFrame

In [ ]:
df[ df['Fare'] > 500  ]

In [ ]:
df[ (df['Fare'] > 500) & (df['Sex'] == 'female') ]

#### Agregar columnas al dataframe

In [ ]:
df['Taxes'] = 5

In [ ]:
df.head(2)

In [ ]:
df['Taxes'] = df['Fare'] * 0.05 + 1

In [ ]:
df.head(2)

In [ ]:
df['Total'] = df['Fare'] + df['Taxes']

In [ ]:
df.head(2)

### Eliminar filas y columnas

In [ ]:
df.drop('Total', axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.drop(0, axis=0, inplace=True)

In [ ]:
df.head()

---

## 3. Del análisis descriptivo a la agregación

Hasta aquí trabajamos principalmente con:
- selección de columnas/filas/celdas,
- estadísticos descriptivos,
- filtrado,
- creación/eliminación de columnas,
- tratamiento inicial de valores faltantes.

Todo lo anterior se conoce como **data preparation / data wrangling**: dejar el dataset en condiciones para responder preguntas.

El siguiente paso natural es **resumir información por grupos** (por ejemplo, por `Sex`, `Pclass`, `Embarked`, rangos de edad, etc.). Para eso usaremos:
- **Multi-Índices**
- **groupby()** y agregaciones avanzadas
- **pivot_table()**, **melt()**

La diferencia principal es el foco:
- antes: operación *fila a fila* o *columna a columna*
- ahora: operación *grupo a grupo*

---


# 4. Agrupamiento de Datos con Titanic

A continuación, aplicamos las mismas herramientas del notebook de agrupamiento, pero usando el dataset **Titanic**.


In [ ]:
# Re-cargamos el dataset para iniciar el bloque de agrupaciones desde una base limpia
# (en la sección anterior hicimos modificaciones educativas al DataFrame)
df = pd.read_csv('titanic.csv')
(df.shape, df.columns)

In [ ]:
df

## 4.1 Multi-índices

Un **MultiIndex** permite indexar un DataFrame por más de una clave. Es útil cuando:
- quieres consultar rápidamente por combinaciones (ej. `Sex` + `Pclass`),
- vas a producir reportes jerárquicos,
- o quieres preparar el dato para ciertas operaciones de agregación.


In [ ]:
# Creamos una vista con multi-índice (no modifica df a menos que lo reasignemos)
df_mi = df.set_index(['Sex','Pclass']).sort_index()
df_mi.head(10)


In [ ]:
# Acceso a una combinación específica de llaves
# Ejemplo: mujeres en 1ra clase
try:
    df_mi.loc[('female', 1)].head()
except KeyError:
    # Si el dataset tiene las llaves con otro formato, mostramos alternativas
    df_mi.index.levels


In [ ]:
df_mi.loc[('female', 1)]

In [ ]:
df_mi.index.names

In [ ]:
df_mi.index.levels

## 4.2 groupby(): el corazón del resumen por grupos

`groupby()` separa el dataset en grupos y luego aplica una o más funciones de agregación.

Ejemplos típicos con Titanic:
- tasa de supervivencia por sexo,
- promedio de tarifa por clase,
- distribución de edades por puerto de embarque.


In [ ]:
# Tasa de supervivencia por sexo (si Survived está en 0/1, el promedio es la tasa)
df.groupby('Sex')['Survived'].mean().sort_values(ascending=False)


In [ ]:
# Supervivencia por (Sexo, Clase) con MultiIndex en el resultado
surv_by_sex_class = df.groupby(['Sex','Pclass'])['Survived'].mean()
surv_by_sex_class


In [ ]:
# Múltiples agregaciones a la vez con .agg()
# - size: número de registros
# - mean Fare: tarifa promedio
# - median Age: mediana de edad
summary = (
    df.groupby(['Sex','Pclass'])
      .agg(n_passengers=('PassengerId','size'),
           surv_rate=('Survived','mean'),
           fare_mean=('Fare','mean'),
           age_median=('Age','median'))
      .sort_values(['surv_rate','n_passengers'], ascending=[False, False])
)
summary


### groupby + transform(): volver del nivel grupo al nivel fila

A veces no basta con la tabla agregada: queremos **traer** una métrica grupal de vuelta al DataFrame para:
- construir features,
- comparar cada fila contra su grupo,
- normalizar por grupo.

`transform()` es la herramienta estándar, porque devuelve una serie con el **mismo largo** que el DataFrame original.


In [ ]:
# Ejemplo: diferencia de la tarifa de cada pasajero respecto al promedio de su clase
# (esto es un feature engineering clásico)
df = df.copy()  # por seguridad
class_fare_mean = df.groupby('Pclass')['Fare'].transform('mean')
df['Fare_vs_ClassMean'] = df['Fare'] - class_fare_mean

(df[['Pclass','Fare','Fare_vs_ClassMean']].head(10))


### groupby + apply(): lógica personalizada por grupo

`apply()` es flexible (permite lógica arbitraria), pero suele ser más lento. Úsalo cuando:
- no puedes expresar la operación con `agg()` / `transform()` / operaciones vectorizadas,
- o cuando necesitas devolver estructuras complejas.


In [ ]:
# Ejemplo: obtener el Top 3 de tarifas por (Sex, Pclass)

def top_fares(group, n=3):
    # Nos quedamos solo con las columnas relevantes para evitar efectos colaterales
    cols = ['Name','Fare','Survived']
    return group[cols].sort_values('Fare', ascending=False).head(n)

Top3 = (
    df[['Sex','Pclass','Name','Fare','Survived']]
      .groupby(['Sex','Pclass'], group_keys=True)
      .apply(lambda g: top_fares(g, n=3))
)

Top3

## 4.3 Pivoteo de tablas

Las tablas pivote son un formato típico de reporte:
- filas = categoría
- columnas = categoría
- valores = métrica agregada

En Pandas, `pivot_table()` es preferible a `pivot()` cuando puede haber duplicados, ya que permite definir la función de agregación.


In [ ]:
# Tasa de supervivencia por Sexo (filas) y Clase (columnas)
pivot_surv = pd.pivot_table(
    df,
    values='Survived',
    index='Sex',
    columns='Pclass',
    aggfunc='mean'
)
pivot_surv


In [ ]:
# Ejemplo con múltiples métricas en pivot_table
pivot_multi = pd.pivot_table(
    df,
    values=['Survived','Fare'],
    index='Embarked',
    columns='Pclass',
    aggfunc={'Survived':'mean','Fare':'mean'}
)
pivot_multi


## 4.4 Despivoteo (melt)

`melt()` convierte datos desde formato "ancho" (muchas columnas) a formato "largo" (columna de variable + columna de valor).

Esto es especialmente útil para:
- alimentar visualizaciones,
- estandarizar estructura para modelado,
- o para merges más simples.

Aquí tomaremos `pivot_surv` y lo llevaremos a formato largo.


In [ ]:
surv_long = (
    pivot_surv
      .reset_index()
      .melt(id_vars='Sex', var_name='Pclass', value_name='SurvivalRate')
      .sort_values(['Sex','Pclass'])
)
surv_long


---

## 5. Cierre

En este flujo conectamos dos ideas:

1) **Preparación / exploración**: entender el dataset, limpiar, filtrar y crear variables.
2) **Agrupación / reporte**: resumir el dataset por categorías para obtener conclusiones accionables.

En la práctica, la preparación define *qué tan confiable* es la agregación. Y la agregación define *qué tan bien* comunicamos hallazgos a negocio.
